In [1]:
from mlflow import MlflowClient
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor

In [2]:
client = MlflowClient(tracking_uri="http://127.0.0.1:8080")

In [14]:
all_experiments = client.search_experiments()

print(all_experiments)

[<Experiment: artifact_location='mlflow-artifacts:/708419498418096915', creation_time=1752759445940, experiment_id='708419498418096915', last_update_time=1752759445940, lifecycle_stage='active', name='Apple_Models', tags={'mlflow.note.content': 'This is the grocery forecasting project. This '
                        'experiment contains the produce models for apples.',
 'project_name': 'grocery-forecasting',
 'project_quarter': 'Q3-2023',
 'store_dept': 'produce',
 'team': 'stores-ml'}>, <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1752746026648, experiment_id='0', last_update_time=1752746026648, lifecycle_stage='active', name='Default', tags={}>]


In [ ]:
default_experiment = [
    {"name": experiment.name, "lifecycle_stage": experiment.lifecycle_stage}
    for experiment in all_experiments
    if experiment.name == "Default"
][0]

pprint(default_experiment)

{'lifecycle_stage': 'active', 'name': 'Default'}


In [6]:
# Provide an Experiment description that will appear in the UI
experiment_description = (
    "This is the grocery forecasting project. "
    "This experiment contains the produce models for apples."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "grocery-forecasting",
    "store_dept": "produce",
    "team": "stores-ml",
    "project_quarter": "Q3-2023",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
produce_apples_experiment = client.create_experiment(
    name="Apple_Models", tags=experiment_tags
)

In [7]:
# Use search_experiments() to search on the project_name tag key

apples_experiment = client.search_experiments(
    filter_string="tags.`project_name` = 'grocery-forecasting'"
)

print(vars(apples_experiment[0]))

{'_experiment_id': '708419498418096915', '_name': 'Apple_Models', '_artifact_location': 'mlflow-artifacts:/708419498418096915', '_lifecycle_stage': 'active', '_tags': {'store_dept': 'produce', 'project_quarter': 'Q3-2023', 'mlflow.note.content': 'This is the grocery forecasting project. This experiment contains the produce models for apples.', 'project_name': 'grocery-forecasting', 'team': 'stores-ml'}, '_creation_time': 1752759445940, '_last_update_time': 1752759445940}


In [8]:
print(apples_experiment[0].tags["team"])

stores-ml


In [9]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def generate_apple_sales_data_with_promo_adjustment(
    base_demand: int = 1000, n_rows: int = 5000
):
    """
    Generates a synthetic dataset for predicting apple sales demand with seasonality
    and inflation.

    This function creates a pandas DataFrame with features relevant to apple sales.
    The features include date, average_temperature, rainfall, weekend flag, holiday flag,
    promotional flag, price_per_kg, and the previous day's demand. The target variable,
    'demand', is generated based on a combination of these features with some added noise.

    Args:
        base_demand (int, optional): Base demand for apples. Defaults to 1000.
        n_rows (int, optional): Number of rows (days) of data to generate. Defaults to 5000.

    Returns:
        pd.DataFrame: DataFrame with features and target variable for apple sales prediction.

    Example:
        >>> df = generate_apple_sales_data_with_seasonality(base_demand=1200, n_rows=6000)
        >>> df.head()
    """

    # Set seed for reproducibility
    np.random.seed(9999)

    # Create date range
    dates = [datetime.now() - timedelta(days=i) for i in range(n_rows)]
    dates.reverse()

    # Generate features
    df = pd.DataFrame(
        {
            "date": dates,
            "average_temperature": np.random.uniform(10, 35, n_rows),
            "rainfall": np.random.exponential(5, n_rows),
            "weekend": [(date.weekday() >= 5) * 1 for date in dates],
            "holiday": np.random.choice([0, 1], n_rows, p=[0.97, 0.03]),
            "price_per_kg": np.random.uniform(0.5, 3, n_rows),
            "month": [date.month for date in dates],
        }
    )

    # Introduce inflation over time (years)
    df["inflation_multiplier"] = (
        1 + (df["date"].dt.year - df["date"].dt.year.min()) * 0.03
    )

    # Incorporate seasonality due to apple harvests
    df["harvest_effect"] = np.sin(2 * np.pi * (df["month"] - 3) / 12) + np.sin(
        2 * np.pi * (df["month"] - 9) / 12
    )

    # Modify the price_per_kg based on harvest effect
    df["price_per_kg"] = df["price_per_kg"] - df["harvest_effect"] * 0.5

    # Adjust promo periods to coincide with periods lagging peak harvest by 1 month
    peak_months = [4, 10]  # months following the peak availability
    df["promo"] = np.where(
        df["month"].isin(peak_months),
        1,
        np.random.choice([0, 1], n_rows, p=[0.85, 0.15]),
    )

    # Generate target variable based on features
    base_price_effect = -df["price_per_kg"] * 50
    seasonality_effect = df["harvest_effect"] * 50
    promo_effect = df["promo"] * 200

    df["demand"] = (
        base_demand
        + base_price_effect
        + seasonality_effect
        + promo_effect
        + df["weekend"] * 300
        + np.random.normal(0, 50, n_rows)
    ) * df[
        "inflation_multiplier"
    ]  # adding random noise

    # Add previous day's demand
    df["previous_days_demand"] = df["demand"].shift(1)
    df["previous_days_demand"].fillna(
        method="bfill", inplace=True
    )  # fill the first row

    # Drop temporary columns
    df.drop(columns=["inflation_multiplier", "harvest_effect", "month"], inplace=True)

    return df

In [10]:
data = generate_apple_sales_data_with_promo_adjustment(base_demand=1_000, n_rows=1_000)

data[-20:]

/tmp/ipykernel_1802930/555690400.py:88: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["previous_days_demand"].fillna(
/tmp/ipykernel_1802930/555690400.py:88: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["previous_days_demand"].fillna(


,date,average_temperature,rainfall,weekend,holiday,price_per_kg,promo,demand,previous_days_demand
980,2025-06-28 14:37:47.844045,34.130183,1.454065,1,0,1.449177,0,1326.306290,1029.418398
981,2025-06-29 14:37:47.844042,32.353643,9.462859,1,0,2.856503,0,1169.129427,1326.306290
982,2025-06-30 14:37:47.844040,18.816833,0.391470,0,0,1.326429,0,990.616709,1169.129427
983,2025-07-01 14:37:47.844037,34.533012,2.120477,0,0,0.970131,0,1068.802075,990.616709
984,2025-07-02 14:37:47.844035,23.057202,2.365705,0,0,1.049931,0,1019.486305,1068.802075
985,2025-07-03 14:37:47.844033,34.810165,3.089005,0,0,2.035149,0,1002.564672,1019.486305
986,2025-07-04 14:37:47.844030,29.208905,3.673292,0,0,2.518098,0,1086.143402,1002.564672
987,2025-07-05 14:37:47.844028,16.428676,4.077782,1,0,1.268979,0,1420.207186,1086.143402
988,2025-07-06 14:37:47.844026,32.067512,2.734454,1,0,0.762317,0,1396.939894,1420.207186
989,2025-07-07 14:37:47.844023,31.938203,13.883486,0,0,1.153301,0,994.409540,1396.939894


In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import mlflow

In [15]:
mlflow.set_tracking_uri("http://127.0.0.1:8080")

def run_experiment_3(n_samples=1000, n_features=5):
    X = np.random.rand(n_samples, n_features)
    noise = np.random.normal(0, 0.1, n_samples)
    y = X[:, 0] + X[:, 1] + noise  # Add noise to make the task realistic

    model = RandomForestRegressor(n_estimators=100, random_state=42)

    with mlflow.start_run(run_name="Experiment 3: Random forest with noise", experiment_id="708419498418096915"):
        model.fit(X, y)
        y_pred = model.predict(X)

        mlflow.log_metric("r2", r2_score(y, y_pred))
        mlflow.log_metric("rmse", root_mean_squared_error(y, y_pred))
        mlflow.log_metric("mae", mean_absolute_error(y, y_pred))

        print("Done: Experiment 3")

run_experiment_3()


Done: Experiment 3
🏃 View run Experiment 3: Random forest with noise at: http://127.0.0.1:8080/#/experiments/708419498418096915/runs/4c2990102d7a43ef9a0a37b90d401b90
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/708419498418096915
